## Example of PySpark ML

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
# Read the dataset

training = spark.read.csv('test5.csv', header=True, inferSchema=True)

In [4]:
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [7]:
# VectorAssembler will make sure that all the features are together grouped in the form of ['Age', 'Experience'] --> new feature
# --> independent feature

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
# inputCols --> which all columns we are taking to group together.
# outputCol --> create a new column out of the inputCols.

featureassembler = VectorAssembler(inputCols=["Age", "Experience"], outputCol="Independent Features")

In [10]:
output = featureassembler.transform(training)

In [11]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data = output.select("Independent Features", "Salary")

In [14]:
finalized_data.show()

# Salary --> dependent feature

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression

In [16]:
# do train test split using randomSplit

train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
# labelCol --> output feature
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor = regressor.fit(train_data)

In [17]:
# Coefficients

regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [18]:
# Intercepts
regressor.intercept

19919.060052212404

In [19]:
# Prediction

pred_results = regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [29.0,4.0]| 20000|19342.03655352618|
+--------------------+------+-----------------+



e:\my_work\practice\pyspark\venv_pyspark\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [21]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(657.9634464738192, 432915.89689570636)